<a href="https://colab.research.google.com/github/JibbyGeorge-DB/HuggingFace/blob/main/HuggingFace_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install or Upgrade transformers

In [ ]:
!pip install transformers

I have connected to T4 instance, lets check the GPU with nvidia-smi

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  if gpu_info.find('Tesla T4') >= 0:
    print("Success - Connected to a T4")
  else:
    print("NOT CONNECTED TO A T4")

Thu Jan  8 21:36:42 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Connecting to Hugguing Face. I have added the HF key in the colab secrets as HF_TOKEN

In [2]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

sdxl-turbo is a very small legacy model.

num_inference_step: is the parameter that determines how many times the model will iteratively refine or "denoise" the image during the generation process.

In [ ]:
#
from IPython.display import display
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo",torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")
prompt = "Vehicles stopped in a traffic light"
image = pipe(prompt=prompt, num_inference_step=4, guidance_scale=0.0).images[0]
display(image)


In [ ]:
# Restart the kernel to bring everything back to normal

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

Trying stable-diffusion-xl-base-1.0 model with 30 inference step

DiffusionPipeline is the Generic Base Class. It is a "smart" loader. When you use DiffusionPipeline.from_pretrained("..."), it looks at the configuration file of the model you're downloading and automatically figures out which specific pipeline class to use (like Stable Diffusion, Flux, or Kandinsky).

In [ ]:
#using pipeline
from diffusers import DiffusionPipeline
import torch

base = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True)
base.to("cuda")
prompt = "Vehicles stopped in a traffic light"

#Generate the image
image = base(
    prompt=prompt,
    num_inference_step=30,
    guidance_scale=0.0
).images[0]

display(image)

In [ ]:
# Restart the kernel to bring everything back to normal
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

StableDiffusionPipeline is a Specific Subclass. It is designed explicitly for the Stable Diffusion v1 and v2 architectures. If you try to load a non-Stable Diffusion model (like a Google Imagen or a Flux model) using this class, it will likely fail because it expects a very specific set of components (a UNet, a VAE, and a specific CLIP text encoder).

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import base64
import os
import io
import IPython.display


# Load the Stable Diffusion pipeline
get_completion = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")

def TTI(prompt):
    output = get_completion(prompt)
    image = output.images[0]
    image = image.resize((512, 512))
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return IPython.display.HTML(f'<img src="data:image/png;base64,{img_str}"/>')


In [ ]:
prompt ="Vehicles stopped in a traffic light"
result = TTI(prompt)
result

In [ ]:
# Restart the kernel to bring everything back to normal
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

https://huggingface.co/black-forest-labs/FLUX.1-schnell

This model dont run on T4, OutOfMemoryError: CUDA out of memory. Tried to allocate 72.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 46.12 MiB is free.

In [3]:
import torch
from diffusers import FluxPipeline
#from diffusers import DiffusionPipeline
from IPython.display import display
from datetime import datetime

start = datetime.now()

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16).to("cuda")
#pipe = DiffusionPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", dtype=torch.bfloat16, device_map="cuda")
generator = torch.Generator(device="cuda").manual_seed(0)
prompt ="Vehicles stopped in a traffic light"


#Generate the image
image = pipe(
    prompt,
    guidance_scale=0.0,
    num_inference_steps=4,
    max_sequence_length=256,
    generator=generator
).images[0]

isplay(image)

stop = datetime.now()

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 72.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 46.12 MiB is free. Process 23568 has 14.69 GiB memory in use. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 7.82 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)